In [138]:

import pandas as pd
import random
import numpy as np
import copy
import networkx as nx
import matplotlib.pyplot as plt

In [139]:
# N-Step SARSA Parameters
numEpisodes = 1000
nStep = 2
discountFactor = 0.9 # TODO: Check if this is a suitable value.

# Env Parameters
cutOffAge = 2
pSwap = 1
pGen= 0.4
maxLinks = 2 # Multiplexing
users = [(1,4)]

initialEdges = {  #(edgeNode1, EdgeNode2): [Age]
    (1, 3): [],  
    (2, 3): [],  
    (3, 4): [],  
    (4, 5): [], 
    (4, 6): [],  
}

In [140]:
class QuantumNetwork:
    def __init__(self, initialEdges, pGen, pSwap, cutOffAge, maxLinks, users):
        self.initialEdges = copy.deepcopy(initialEdges)
        self.currentEdges = {}
        self.pGen = pGen
        self.pSwap = pSwap
        self.cutOffAge = cutOffAge
        self.maxLinks = maxLinks
        self.users= users
        
    def checkEndToEndEntanglement(self): # I will need to change in the future, currently just for testing.
        seenEndToEnd = False
        for edge in self.currentEdges:
            if edge in self.users:
                print(f'End to end entanglement found for {edge}')
                seenEndToEnd = True
        if not(seenEndToEnd):
            print("No end to end entanglements seen")

    def reset(self):
        # Reset currentEdges to be empty
        self.currentEdges = {}
        print("Quantum network has been reset to the initial state.")

    def increaseAgeDiscardOld(self):
        print('Increasing time by 1')
        edges_to_delete = []
        for edge, ages in list(self.currentEdges.items()):
            new_ages = [age + 1 for age in ages if age + 1 < self.cutOffAge]
            if len(ages) != len(new_ages): # Just for logging.
                print(f'Discarded {len(ages) - len(new_ages)} links due to old age cutoff.')
            if new_ages:
                self.currentEdges[edge] = new_ages
            else:
                edges_to_delete.append(edge)

        for edge in edges_to_delete:
            del self.currentEdges[edge]
            print(f"Edge {edge} removed from currentEdges due to lack of entanglements.")

    def discardEntanglement(self, edge):
        if edge in self.currentEdges:
            ages = self.currentEdges[edge]
            if ages:
                removed_age = ages.pop(0)  # Remove the oldest entanglement
                print(f"Discarded entanglement of age {removed_age} from edge {edge}.")
                if not ages:
                    del self.currentEdges[edge]
                    print(f"Edge {edge} removed from currentEdges as it has no entanglements left.")

    def attemptEntanglementGeneration(self):
        print("\nAttempting Entanglement Generation:")
        for edge in self.initialEdges:
            if edge in self.currentEdges:
                if len(self.currentEdges[edge]) < self.maxLinks:
                    if random.random() <= self.pGen:
                        self.currentEdges[edge].append(0)
                        print(f"Generated entanglement for edge {edge}. Updated ages: {self.currentEdges[edge]}")
                    else:
                        print(f"No entanglement generated for edge {edge} this attempt.")
                else:
                    print(f"Edge {edge} has reached the maximum number of links ({self.maxLinks}).")
            else:
                if random.random() <= self.pGen:
                    self.currentEdges[edge] = [0]
                    print(f"Generated entanglement for edge {edge}.")
                else:
                    print(f"No entanglement generated for edge {edge} this attempt.")

    def performSwapping(self, edge1, edge2):
        combined_nodes = list(edge1) + list(edge2)
        unique_nodes = sorted([node for node in combined_nodes if combined_nodes.count(node) == 1])

        if len(unique_nodes) != 2:
            print("Invalid edges for swapping: cannot determine new edge.")
            return

        newLink = tuple(unique_nodes)
        old_ages = self.currentEdges.get(edge1, []) + self.currentEdges.get(edge2, [])
        newAge = max(old_ages) if old_ages else 0
        print(f"Creating new edge {newLink} with entanglement age {newAge}.")

        # Add or update the new link with the new entanglement
        if newLink in self.currentEdges:
            if len(self.currentEdges[newLink]) < self.maxLinks:
                self.currentEdges[newLink].append(newAge)
                print(f"Added entanglement to existing edge {newLink}. Updated ages: {self.currentEdges[newLink]}")
            else:
                print(f"Cannot add entanglement to edge {newLink}; maximum links reached.")
        else:
            self.currentEdges[newLink] = [newAge]
            print(f"Created new edge {newLink} with entanglement age {newAge}.")

        # Discard one entanglement from each old edge
        self.discardEntanglement(edge1)
        self.discardEntanglement(edge2)

    def attemptSwapping(self, edge1, edge2):
        print(f"\nAttempting Swapping between {edge1} and {edge2}:")
        if edge1 not in self.currentEdges or edge2 not in self.currentEdges:
            print("One or both edges do not exist in currentEdges.")
            return

        if not self.currentEdges[edge1] or not self.currentEdges[edge2]:
            print("One or both edges do not have entanglement to swap.")
            return

        if random.random() <= self.pSwap:
            self.performSwapping(edge1, edge2)
        else:
            print('Swapping not performed based on pSwap probability.')

    def showMatrix(self):
        print("\nCurrent Entanglement Matrix:")
        for edge, ages in self.currentEdges.items():
            print(f"Edge {edge}: Ages {ages}")

    def showGraph(self):
        G = nx.Graph()

        # Add edges with multiple entanglements
        for edge, ages in self.currentEdges.items():
            for age in ages:
                G.add_edge(edge[0], edge[1], age=age)

        pos = nx.spring_layout(G)  # Spring layout for better visualization

        # Draw nodes
        nx.draw_networkx_nodes(G, pos, node_size=700, node_color="skyblue")
        # Draw edges
        nx.draw_networkx_edges(G, pos, width=2, alpha=0.7, edge_color="black")
        # Draw labels for nodes
        nx.draw_networkx_labels(G, pos, font_size=12, font_color="black")
        # Draw edge labels for age
        edge_labels = {(u, v): f"Age: {data['age']}" for u, v, data in G.edges(data=True)}
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

        # Display the graph
        plt.title("Quantum Network Graph")
        plt.axis("off")
        plt.show()


In [141]:
random.seed(27)
network = QuantumNetwork(initialEdges, pGen, pSwap, cutOffAge, maxLinks, users)

print('####################################################')
network.attemptEntanglementGeneration()
network.showMatrix()
print('####################################################')

network.increaseAgeDiscardOld()
network.showMatrix()
print('####################################################')

network.attemptEntanglementGeneration()
network.attemptEntanglementGeneration()
network.showMatrix()
print('####################################################')
network.increaseAgeDiscardOld()
network.showMatrix()
print('####################################################')

####################################################

Attempting Entanglement Generation:
No entanglement generated for edge (1, 3) this attempt.
No entanglement generated for edge (2, 3) this attempt.
No entanglement generated for edge (3, 4) this attempt.
Generated entanglement for edge (4, 5).
Generated entanglement for edge (4, 6).

Current Entanglement Matrix:
Edge (4, 5): Ages [0]
Edge (4, 6): Ages [0]
####################################################
Increasing time by 1

Current Entanglement Matrix:
Edge (4, 5): Ages [1]
Edge (4, 6): Ages [1]
####################################################

Attempting Entanglement Generation:
No entanglement generated for edge (1, 3) this attempt.
Generated entanglement for edge (2, 3).
Generated entanglement for edge (3, 4).
No entanglement generated for edge (4, 5) this attempt.
Generated entanglement for edge (4, 6). Updated ages: [1, 0]

Attempting Entanglement Generation:
No entanglement generated for edge (1, 3) this attempt.
No e

In [142]:
random.seed(27)
network = QuantumNetwork(initialEdges, pGen, pSwap, cutOffAge, maxLinks, users=[(4,6)])
network.attemptEntanglementGeneration()
network.showMatrix()
print('####################################################')
network.performSwapping((4,5), (5,6))
network.showMatrix()
network.checkEndToEndEntanglement()



Attempting Entanglement Generation:
No entanglement generated for edge (1, 3) this attempt.
No entanglement generated for edge (2, 3) this attempt.
No entanglement generated for edge (3, 4) this attempt.
Generated entanglement for edge (4, 5).
Generated entanglement for edge (4, 6).

Current Entanglement Matrix:
Edge (4, 5): Ages [0]
Edge (4, 6): Ages [0]
####################################################
Creating new edge (4, 6) with entanglement age 0.
Added entanglement to existing edge (4, 6). Updated ages: [0, 0]
Discarded entanglement of age 0 from edge (4, 5).
Edge (4, 5) removed from currentEdges as it has no entanglements left.

Current Entanglement Matrix:
Edge (4, 6): Ages [0, 0]
End to end entanglement found for (4, 6)
